# Implement user authentication and authorization: coding walkthrough

This notebook demonstrates the hands-on coding steps from `notes.md`, showing how to register a Microsoft Entra application, authenticate with MSAL, and explore Microsoft Graph in a classroom-friendly way.

## Prerequisites
- An Azure subscription and a registered application with a redirect URI of `http://localhost`.
- The Application (client) ID and Directory (tenant) ID recorded from the Azure portal.
- Local Python 3.8+ environment with browser access (interactive login opens a new tab).

> **Security reminder:** never hard-code secrets in notebooks. Store them in environment variables or a `.env` file that stays out of source control.

In [1]:
%pip install --upgrade msal python-dotenv azure-identity msgraph-sdk azure-storage-blob requests

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached azure_identity-1.25.0-py3-none-any.whl.metadata (87 kB)
  Using cached azure_identity-1.25.0-py3-none-any.whl.metadata (87 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached cryptography-46.0.1-cp311-abi3-macosx_10_9_universal2.whl.metadata (5.7 kB)
  Using cached cryptography-46.0.1-cp311-abi3-macosx_10_9_universal2.whl.metadata (5.7 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached charset_normalizer-3.4.3-cp312-cp312-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py

## Configure environment values
1. Create a `.env` file next to this notebook (if it does not exist).
2. Add `CLIENT_ID` and `TENANT_ID` from your app registration.
3. Run the cell below to load those values and optionally prompt for them.

The helper will persist any missing values back to the `.env` file so future sessions can reuse them.

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv, set_key

ENV_PATH = Path('.env')
if not ENV_PATH.exists():
    ENV_PATH.touch()
    print(f'Created blank {ENV_PATH.resolve()} - remember to exclude it from source control.')

load_dotenv(ENV_PATH)


def ensure_setting(env_path: Path, name: str, prompt: str, default: str | None = None) -> str:
    value = os.getenv(name, '').strip()
    if not value:
        user_input = input(f'{prompt.strip()} ').strip()
        value = user_input or (default.strip() if default else '')
        if value:
            set_key(env_path, name, value)
            print(f'Saved {name} to {env_path}')
    if value:
        hint = value if len(value) <= 8 else f"{value[:4]}...{value[-4:]}"
        print(f'{name} loaded ({hint})')
    else:
        print(f'{name} is missing. Update {env_path} before continuing.')
    return value

CLIENT_ID = ensure_setting(ENV_PATH, 'CLIENT_ID', 'Enter your Application (client) ID:')
TENANT_ID = ensure_setting(
    ENV_PATH,
    'TENANT_ID',
    "Enter your Directory (tenant) ID (or leave blank for 'common'):",
    default='common'
) or 'common'
STORAGE_ACCOUNT_NAME = ensure_setting(
    ENV_PATH,
    'STORAGE_ACCOUNT_NAME',
    'Enter your Storage account name (optional for SAS demos):'
)
STORAGE_ACCOUNT_KEY = ensure_setting(
    ENV_PATH,
    'STORAGE_ACCOUNT_KEY',
    'Enter your Storage account key (optional—press Enter to skip):'
)

if not CLIENT_ID:
    raise ValueError('CLIENT_ID is required to proceed.')

print(f'Using authority https://login.microsoftonline.com/{TENANT_ID}')

## Acquire tokens with MSAL (PublicClientApplication)
This section mirrors the console app from the notes: we define scopes, build a `PublicClientApplication`,
and request tokens silently when possible, falling back to interactive sign-in. Tokens are cached on disk
so subsequent runs demonstrate silent acquisition.

In [ ]:
from datetime import datetime
from pathlib import Path
import msal

# Request minimal profile access up front
SCOPES = ['User.Read']

cache_path = Path('.msal_cache.json')
token_cache = msal.SerializableTokenCache()
if cache_path.exists():
    token_cache.deserialize(cache_path.read_text())

app = msal.PublicClientApplication(
    CLIENT_ID,
    authority=f'https://login.microsoftonline.com/{TENANT_ID}',
    token_cache=token_cache
)

def acquire_user_token(scopes: list[str]) -> dict:
    accounts = app.get_accounts()
    if accounts:
        result = app.acquire_token_silent(scopes, account=accounts[0])
        if result:
            print(f"✅ Reused cached token for {accounts[0]['username']}")
            return result
    print('🔐 No cached token available. Launching interactive sign-in...')
    return app.acquire_token_interactive(scopes=scopes)

auth_result = acquire_user_token(SCOPES)

if token_cache.has_state_changed:
    cache_path.write_text(token_cache.serialize())
    print(f'Token cache updated at {cache_path.resolve()}')

if 'access_token' not in auth_result:
    raise RuntimeError(auth_result.get('error_description', 'Authentication failed'))

expires_local = datetime.fromtimestamp(int(auth_result['expires_on']))
print(f"Token acquired for {auth_result['account']['username']}")
print(f'Expires on {expires_local:%Y-%m-%d %H:%M:%S} local time')

## Inspect token metadata (no secrets shown)
This step helps understand the structure of a JWT access token by decoding the header and payload
sections locally. Only claims are displayed—never share the raw token.

In [ ]:
import base64
import json

def decode_jwt_part(token: str, index: int) -> dict:
    parts = token.split('.')
    if len(parts) <= index:
        raise ValueError('Unexpected token format')
    segment = parts[index]
    padding = '=' * (-len(segment) % 4)
    decoded_bytes = base64.urlsafe_b64decode(segment + padding)
    return json.loads(decoded_bytes)

header = decode_jwt_part(auth_result['access_token'], 0)
payload = decode_jwt_part(auth_result['access_token'], 1)

print('Header:', header)
print('Audience:', payload.get('aud'))
print('Scopes (scp):', payload.get('scp'))
print('Issued for appId:', payload.get('appid'))
print('Token lifetime (s):', payload.get('exp') - payload.get('iat'))

## Call Microsoft Graph with the acquired token
Use the REST API directly to retrieve profile information from `/me`. This highlights how to attach
the bearer token in the `Authorization` header.

In [ ]:
import requests
from pprint import pprint

GRAPH_BASE = 'https://graph.microsoft.com/v1.0'

def call_graph(access_token: str, endpoint: str, params: dict | None = None) -> dict:
    response = requests.get(
        f'{GRAPH_BASE}{endpoint}',
        headers={'Authorization': f'Bearer {access_token}'},
        params=params
    )
    if response.status_code == 401:
        raise RuntimeError('The token expired. Re-run the authentication cell.')
    response.raise_for_status()
    return response.json()

profile = call_graph(auth_result['access_token'], '/me')
pprint({
    'displayName': profile.get('displayName'),
    'userPrincipalName': profile.get('userPrincipalName'),
    'id': profile.get('id'),
    'jobTitle': profile.get('jobTitle'),
})

## Demonstrate incremental consent
Requesting additional permissions later keeps the initial consent dialog short. The following cell asks for
`Mail.Read` on top of `User.Read`, prompting only if the scope was not previously granted.

In [ ]:
MAIL_SCOPES = ['Mail.Read']
EXTENDED_SCOPES = list(dict.fromkeys(SCOPES + MAIL_SCOPES))

mail_result = app.acquire_token_silent(EXTENDED_SCOPES, account=app.get_accounts()[0]) if app.get_accounts() else None
if not mail_result:
    print('Requesting incremental consent for Mail.Read...')
    mail_result = app.acquire_token_interactive(scopes=EXTENDED_SCOPES)

if token_cache.has_state_changed:
    cache_path.write_text(token_cache.serialize())

if 'access_token' not in mail_result:
    raise RuntimeError(mail_result.get('error_description', 'Failed to obtain extended scopes'))

mail_token = mail_result['access_token']
print('Extended scopes granted:', mail_result.get('scope'))

### Query messages with filtering
With `Mail.Read` consent granted, retrieve the most recent messages and show how to apply OData query options.
Handle empty mailboxes or restricted accounts gracefully.

In [ ]:
params = {
    '$select': 'subject,sender,receivedDateTime',
    '$orderby': 'receivedDateTime DESC',
    '$top': 5
}
messages = call_graph(mail_token, '/me/messages', params=params)
items = messages.get('value', [])
if not items:
    print('No messages returned. Ensure the mailbox contains mail and the account has Mail.Read consent.')
else:
    for message in items:
        sender = (message.get('sender') or {}).get('emailAddress', {}).get('address')
        received = message.get('receivedDateTime')
        subject = message.get('subject') or '(no subject)'
        print(f"{received} | {sender} | {subject}")

## Manage shared access signatures for Azure Storage
The notes also cover shared access signatures (SAS) and stored access policies. The next few cells show how to
connect by using the Azure Storage SDK for Python, generate a service SAS for blobs, and configure a stored
access policy that you can reference from SAS tokens. These examples expect `STORAGE_ACCOUNT_NAME` and
`STORAGE_ACCOUNT_KEY` to be present in the `.env` file; skip the cells or mock the responses if you're
presenting without a live storage account.

In [ ]:
from datetime import datetime, timedelta, timezone
from azure.core.exceptions import ResourceExistsError
from azure.storage.blob import (
    BlobServiceClient,
    BlobSasPermissions,
    generate_blob_sas,
    AccessPolicy,
    ContainerSasPermissions
)

if not (STORAGE_ACCOUNT_NAME and STORAGE_ACCOUNT_KEY):
    raise RuntimeError('Set STORAGE_ACCOUNT_NAME and STORAGE_ACCOUNT_KEY in .env to run the SAS samples.')

blob_service_client = BlobServiceClient(
    account_url=f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net",
    credential=STORAGE_ACCOUNT_KEY
)

container_name = "demo-container"
print(f"Connected to storage account '{STORAGE_ACCOUNT_NAME}'")

container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
    print(f"Created container: {container_name}")
except ResourceExistsError:
    print(f"Container already exists: {container_name}")

In [ ]:
blob_name = "hello-demo.txt"

# Upload sample content so the SAS has something to reference
blob_client = container_client.get_blob_client(blob_name)
blob_client.upload_blob("Hello from SAS demo", overwrite=True)

expiry_time = datetime.now(timezone.utc) + timedelta(hours=1)

blob_sas_token = generate_blob_sas(
    account_name=STORAGE_ACCOUNT_NAME,
    container_name=container_name,
    blob_name=blob_name,
    account_key=STORAGE_ACCOUNT_KEY,
    permission=BlobSasPermissions(read=True, write=False, add=False, create=False, delete=False),
    expiry=expiry_time,
    start=datetime.now(timezone.utc) - timedelta(minutes=5),
    protocol="https"
 )

print("Sample SAS token (truncated):", blob_sas_token[:50] + "...")

print("Use with URI:")

print(f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net/{container_name}/{blob_name}?{blob_sas_token}")

In [ ]:
policy_id = "demo-policy"

policy = AccessPolicy(
    permission=ContainerSasPermissions(read=True, write=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=2),
    start=datetime.now(timezone.utc)

)

container_client.set_container_access_policy(
    signed_identifiers={policy_id: policy}

)

print(f"Stored access policy '{policy_id}' set on {container_name}")

In [ ]:
policy_sas_token = generate_blob_sas(
    account_name=STORAGE_ACCOUNT_NAME,
    container_name=container_name,
    blob_name=blob_name,
    account_key=STORAGE_ACCOUNT_KEY,
    policy_id=policy_id
 )

print("SAS token using stored access policy (truncated):", policy_sas_token[:50] + "...")

### Optional: create the same policy from the Azure CLI
Run the following command from a terminal (not within the notebook) if you prefer using the Azure CLI. Update
the placeholders to match your resource names:
```bash
az storage container policy create \
    --name demo-policy \
    --container-name demo-container \
    --start 2025-01-01T00:00Z \
    --expiry 2025-01-01T08:00Z \
    --permissions acdwr \
    --account-name <storage-account-name> \
    --account-key <storage-account-key>
```

## Use the Microsoft Graph SDK with Device Code flow
The SDK wraps REST calls and handles pagination, retries, and serialization. Device Code flow works well for
shared demos or environments where a browser is unavailable. Learners can contrast this experience with
the direct REST call above.

In [ ]:
from azure.identity import DeviceCodeCredential
from msgraph.core import GraphClient

def prompt_device_code(device_code: dict):
    print(device_code['message'])

device_credential = DeviceCodeCredential(
    client_id=CLIENT_ID,
    tenant_id=TENANT_ID,
    prompt_callback=prompt_device_code
)
sdk_client = GraphClient(credential=device_credential, scopes=EXTENDED_SCOPES)

sdk_profile = sdk_client.get('/me').json()
print('Graph SDK retrieved:', sdk_profile.get('displayName'), sdk_profile.get('userPrincipalName'))

## Wrap-up
- MSAL manages token acquisition, caching, and interactive flows with a few lines of code.
- Incremental consent keeps the initial permission prompt focused, requesting broader access only when a feature needs it.
- Microsoft Graph can be called directly via HTTP, through the SDK, or by using device code flow—choose the approach that best fits your scenario.
- Shared access signatures give time-bound permissions to storage resources; stored access policies make it easy to rotate or revoke that access centrally.
- Remember to sign out users in the browser if multiple students reuse the same machine, and to remove cached tokens when the demo is complete.

Further reading: [Microsoft identity platform documentation](https://learn.microsoft.com/azure/active-directory/develop/), [Microsoft Graph REST API reference](https://learn.microsoft.com/graph/api/overview), and [Shared access signatures overview](https://learn.microsoft.com/azure/storage/common/storage-sas-overview).